In [1]:
###Loading packages
import numpy as np
import pandas as pd
import math
import itertools
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
from keras.callbacks import Callback, ModelCheckpoint
from sklearn.utils import class_weight

from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, f1_score

from keras import backend as K
import tensorflow as tf

from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

Using TensorFlow backend.


In [2]:
###create model
def create_model(optimizer, activation):
    
    ###define model
    model = Sequential()
    model.add(Dense(512, activation=activation,input_shape=(978,)))
    model.add(Dense(256, activation=activation))
    model.add(Dense(128, activation=activation))
    model.add(Dense(64, activation=activation))
    model.add(Dense(32, activation=activation))
    model.add(Dense(16, activation=activation))
    model.add(Dense(8, activation=activation))
    model.add(Dense(1, activation='sigmoid'))
    
    ###compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy',monitor_f])

    return model


###fit model
def fit_model(X_train, y_train, X_test, y_test,n,model_path,model):
    ###balanced class weight
    class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train),y_train)
    ###define checkpoint for the best model
    checkpoint = ModelCheckpoint(model_path, verbose=1, monitor='val_monitor_f',save_best_only=True, mode='max')
    ###fit model
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=n, class_weight=class_weights,callbacks=[checkpoint])


###predict the independent validation set
def predict_validation(X, y, model_path,model,optimizer):
    model.load_weights(model_path)
    model.compile(loss='binary_crossentropy',optimizer=optimizer)
    y_pred_class = model.predict_classes(X)
    y_pred = model.predict(X)
    result = measurements(y, y_pred_class, y_pred)
    return result


In [3]:
def measurements(y_test, y_pred, y_pred_prob):  
    acc = metrics.accuracy_score(y_test, y_pred)
    sensitivity = metrics.recall_score(y_test, y_pred)
    TN, FP, FN, TP = confusion_matrix(y_test, y_pred).ravel()
    specificity = TN/(TN+FP)
    precision = metrics.precision_score(y_test, y_pred)
    npv = TN/(TN+FN)
    mcc = metrics.matthews_corrcoef(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_prob)
    f1 = metrics.f1_score(y_test, y_pred)
    return [auc, sensitivity, specificity, acc, f1, mcc, precision, npv]

In [4]:
def evaluation(y_true, y_pred):
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos

    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos

    TP = K.sum(y_pos * y_pred_pos)
    TN = K.sum(y_neg * y_pred_neg)

    FP = K.sum(y_neg * y_pred_pos)
    FN = K.sum(y_pos * y_pred_neg)
    return TP, TN, FP, FN

In [5]:
###monitor function for finding the best models
def monitor_f(y_true, y_pred):
    TP, TN, FP, FN = evaluation(y_true, y_pred)
    part_a = 0.05*(TN/(TN+FP))
    part_b = ((TP*TN*TN)-(FP*FN*TN))/((K.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)))*(TN+FP))
    return part_a + part_b 

In [6]:
def print_result(model_name, purpose, result):
    print('\033[1mOptimized {} model {} performance: \033[0m'.format(model_name, purpose))
    print("AUC:         {0:.3f}".format(result[0]))
    print("Sensitivity: {0:.3f}".format(result[1]))
    print("Specificity: {0:.3f}".format(result[2]))
    print("Accuracy:    {0:.3f}".format(result[3]))
    print("F1:          {0:.3f}".format(result[4]))
    print("MCC:         {0:.3f}".format(result[5]))
    print("PPV:         {0:.3f}".format(result[6]))
    print("NPV:         {0:.3f}".format(result[7]))

### Import dataset
#### Data is used for  training and validation
#### Test is only used for testing

In [7]:
data = pd.read_csv(r'''C:\Users\Ting.Li\Documents\2019\projects\L1000\data\github\data.csv''', low_memory=False)
test = pd.read_csv(r'''C:\Users\Ting.Li\Documents\2019\projects\L1000\data\github\testing.csv''', low_memory=False)

In [8]:
X = data.iloc[:,3:].values
y = data.loc[:,'DILIst.1'].values

### Parameter tuning

In [ ]:
col_names =  ['Group', 'model','auc', 'sensitivity','specificity', 'acc', 'f1', 'mcc','precision', 'npv']

In [ ]:
optimizers = ['SGD', 'Adam', 'RMSProp', 'Adadelta']
activations = ['relu', 'elu', 'selu', 'tanh']
paras = [' '.join(l) for l in itertools.product(optimizers, activations)]

col_names =  ['Group', 'model','sensitivity','specificity', 'precision', 'acc', 'mcc', 'auc']
training_metrics  = pd.DataFrame(columns = col_names)

In [ ]:
model_root_path = r'''C:\Users\Ting.Li\Documents\2019\projects\L1000\data\github\best_model_'''

In [ ]:
for i in range(len(paras)):
    optimizer, activation = paras[i].split(' ')[0], paras[i].split(' ')[1]
    name = 'DNN_'+'paras_'+str(i)+'_optimizer_'+optimizer+'_activation_'+activation
    for j in range(100):
        X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y,test_size=0.2, random_state=j)
        model_path = model_root_path + name +'_seed_'+ str(j)+ '_weights.h5'
        model = create_model(optimizer, activation)
        fit_model(X_train, y_train, X_test, y_test,30,model_path,model)
        result = predict_validation(X_test, y_test, model_path, model, optimizer)
        training_metrics.loc[len(training_metrics)] = [str(j), name, result[0], result[1], result[2], result[3],  result[4], result[5]]
training_metrics.to_csv(r'''C:\Users\Ting.Li\Documents\2019\projects\L1000\data\github\dnn_training_metrics_all.csv''')

### Optimized model

In [9]:
model = create_model('Adam', 'elu')
### Load the optimized model weights
model_path = r'''C:\Users\Ting.Li\Documents\2019\projects\L1000\data\github\best_model_DNN_paras_5_paras_optimizer_Adam_activation_elu_seed_93_weights.h5'''
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=93)
### Optimized DNN model training performance
print_result('DNN', 'training', predict_validation(X_test, y_test, model_path, model, 'Adam'))
### Optimized DNN model testing performance
print_result('DNN', 'testing', predict_validation(test.iloc[:, 3:].values, test.loc[:,'DILIst.1'].values, model_path, model, 'Adam'))

Optimized DNN model training performance: 
AUC:         0.802
Sensitivity: 0.851
Specificity: 0.630
Accuracy:    0.761
F1:          0.809
MCC:         0.497
PPV:         0.771
NPV:         0.742
Optimized DNN model testing performance: 
AUC:         0.798
Sensitivity: 0.839
Specificity: 0.603
Accuracy:    0.743
F1:          0.795
MCC:         0.458
PPV:         0.756
NPV:         0.718
